<a href="https://colab.research.google.com/github/T-mang/hello/blob/master/spark_rdd_part0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jre-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15 http

In [5]:
# spark job
import pyspark

from pyspark import SparkConf
from pyspark import SparkContext
conf = SparkConf()
#set to run job in local mode
#conf.setMaster('local')
#conf.setAppName('spark-basic')
#sc = SparkContext(conf=conf)
#find sc from spark session
sc = spark.sparkContext

def mod(x):
  import numpy as np
  return (x, np.mod(x,2))

rdd = sc.parallelize(range(1000)).map(mod).take(10)  
print(rdd)

[(0, 0), (1, 1), (2, 0), (3, 1), (4, 0), (5, 1), (6, 0), (7, 1), (8, 0), (9, 1)]


In [9]:
lst = [1,2,3,4,5,6]
rdd = sc.parallelize(lst).take(3)
print(rdd)

[1, 2, 3]


In [10]:
from google.colab import files
uploaded = files.upload()


Saving original.csv to original.csv


In [ ]:
textFile = sc.textFile("original.csv")
textFile.persist()

In [ ]:
#cache is fault tolerant
textFile = sc.textFile("original.csv")
textFile.cache()

In [11]:
rdd = sc.parallelize([2,3,4])
sorted(rdd.flatMap(lambda x: range(1,x)).collect())

[1, 1, 1, 2, 2, 3]

In [12]:
rdd = sc.parallelize([1,2,3,4,5,6])
rdd.filter (lambda x: x % 2 == 0).collect()

[2, 4, 6]

In [13]:
rdd = sc.parallelize(["spark","rdd","example","sample", "example"])
rdd2 = rdd.map(lambda x:(x, 1))
rdd2.collect()

[('spark', 1), ('rdd', 1), ('example', 1), ('sample', 1), ('example', 1)]

In [15]:
#sample

parallel = sc.parallelize(range(9))
parallel.sample(True,.2).count()

2

In [16]:
parallel.sample(False,1).collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [17]:
#Union
parallel = sc.parallelize(range(1,9))
par = sc.parallelize(range(5,15))
parallel.union(par).collect()

[1, 2, 3, 4, 5, 6, 7, 8, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [18]:
#Intersection
parallel = sc.parallelize(range(1,9))
par = sc.parallelize(range(5,15))
parallel.intersection(par).collect()

[8, 5, 6, 7]

In [19]:
#distinct
parallel = sc.parallelize(range(1,9))
par = sc.parallelize(range(5,15))
parallel.union(par).distinct().collect()

[4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11]

In [20]:
#sortBy
y = sc.parallelize([5,7,9,1,4,6,3,2])
y.sortBy(lambda c: c, True).collect()

[1, 2, 3, 4, 5, 6, 7, 9]

In [22]:
#desc order
z = sc.parallelize([("H", 1),("A", 26),("Z", 10),("L", 5)])
z.sortBy(lambda c: c, False).collect()

[('Z', 10), ('L', 5), ('H', 1), ('A', 26)]

In [23]:
#mapPartitions
#can be used as alternative to map() and foreach(), 
#called once for each partition, unlike map() and 
#foreach() called for each element in rdd
rdd = sc.parallelize([1,2,3,4],2)
def f(iterator): yield sum(iterator)
rdd.mapPartitions(f).collect()

[3, 7]

In [24]:
#withIndex
rdd = sc.parallelize([1,2,3,4],4)
def f(splitIndex, iterator): yield splitIndex
rdd.mapPartitionsWithIndex(f).sum()


6

In [25]:
#GroupBy
y = sc.parallelize([5,7,9,1,4,6,3,2,8])
result = y.groupBy(lambda x: x % 2).collect()
sorted([(x, sorted(y)) for (x,y) in result])

[(0, [2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [27]:
#keyBy
x = sc.parallelize(range(1,9)).keyBy(lambda x: x*x)
y = sc.parallelize(zip(range(0,5),range(0,5)))
[(x, list(map(list, y))) for x,y in sorted(x.cogroup(y).collect())]

[(0, [[], [0]]),
 (1, [[1], [1]]),
 (2, [[], [2]]),
 (3, [[], [3]]),
 (4, [[2], [4]]),
 (9, [[3], []]),
 (16, [[4], []]),
 (25, [[5], []]),
 (36, [[6], []]),
 (49, [[7], []]),
 (64, [[8], []])]

In [28]:
#zip
x = sc.parallelize(range(0,5))
y = sc.parallelize(range(1000, 1005))
x.zip(y).collect()

[(0, 1000), (1, 1001), (2, 1002), (3, 1003), (4, 1004)]

In [29]:
#zip withIndex
sc.parallelize(["a","b","c","d"], 3).zipWithIndex().collect()

[('a', 0), ('b', 1), ('c', 2), ('d', 3)]

In [30]:
#repartition
rdd = sc.parallelize([5,7,9,1,4,6,3,2,8],4)
sorted(rdd.glom().collect())

[[3, 2, 8], [4, 6], [5, 7], [9, 1]]

In [31]:
len(rdd.repartition(2).glom().collect())

2

In [32]:
#coalesce
sc.parallelize([5,7,9,1,4,6,3,2,8],3).glom().collect()

[[5, 7, 9], [1, 4, 6], [3, 2, 8]]

In [33]:
sc.parallelize([5,7,9,1,4,6,3,2,8],3).coalesce(2).glom().collect()

[[5, 7, 9], [1, 4, 6, 3, 2, 8]]

In [34]:
#reduce
from operator import add
sc.parallelize([1,2,3,4,5]).reduce(add)

15

In [36]:
sc.parallelize((2 for _ in range(10))).map(lambda x: 1).cache().reduce(add)

10

In [38]:
#first
sc.parallelize([2,3,4]).first()

2

In [39]:
#takeOrdered
nums = sc.parallelize([1,6, 7,2,3,8,4,5])
nums.takeOrdered(5)

[1, 2, 3, 4, 5]

In [40]:
#count
nums = sc.parallelize([1,6, 7,2,3,8,4,5])
nums.count()

8

In [43]:
#saveAsTextFile -full path
nums = sc.parallelize([1,6, 7,2,3,8,4,5])
nums.saveAsTextFile("mydata.txt")

In [44]:
#forEach
def f(x): print(x)
sc.parallelize([1,2,3,4,5]).foreach(f)

In [47]:
#foreach partition
def f(iterator):
  for x in iterator:
    print(x)
sc.parallelize([1,2,3,4]).foreachPartition(f)   

In [48]:
#Mathematical actions
numbers = sc.parallelize(range(1,100))
numbers.sum()

4950

In [49]:
numbers.variance()

816.6666666666666

In [50]:
numbers.max()

99

In [51]:
numbers.mean()

50.0

In [52]:
numbers.stdev()

28.577380332470412

In [54]:
#creating pair rdds
rdd = sc.parallelize([("a1","b1","c1","d1","e1"),("a2","b2","c2","d2","e2")])
result = rdd.map(lambda x: (x[0], list(x[1:])))
result.collect()

[('a1', ['b1', 'c1', 'd1', 'e1']), ('a2', ['b2', 'c2', 'd2', 'e2'])]

In [59]:
#word count using rdd
rdd = sc.textFile("word_count.text")
nonempty_lines = rdd.filter(lambda x: len(x) > 0)
words = nonempty_lines.flatMap(lambda x: x.split(' '))
wordcount = words.map(lambda x:(x,1)).reduceByKey(lambda x,y: x+y).map(lambda x: (x[1],x[0])).sortByKey(False)
wrdcnt =wordcount.collect()
#for word in wordcount.collect():
for word in wrdcnt:
  print(word)
wordcount.saveAsTextFile("Wordcount")  
#wrdcnt.saveAsFile("Wordcount")  


(71, 'the')
(33, 'of')
(21, 'in')
(21, 'and')
(20, 'New')
(17, 'York')
(17, 'to')
(11, 'a')
(10, 'The')
(8, 'was')
(8, 'from')
(7, 'City')
(6, 'became')
(5, 'state')
(5, 'first')
(4, 'around')
(4, 'as')
(4, 'United')
(3, 'trade')
(3, 'major')
(3, 'during')
(3, 'American')
(3, 'States')
(3, 'Great')
(3, 'opened')
(3, 'In')
(3, 'British')
(3, 'were')
(3, 'for')
(3, 'with')
(3, 'capital')
(3, 'century,')
(3, 'most')
(2, 'European')
(2, 'led')
(2, 'came')
(2, 'colony')
(2, 'II')
(2, 'war.')
(2, 'Liberty')
(2, 'at')
(2, 'point')
(2, 'Constitution.')
(2, 'significant')
(2, 'transportation')
(2, '19th')
(2, 'service')
(2, 'before')
(2, 'World')
(2, 'Native')
(2, "York's")
(2, 'main')
(2, 'by')
(2, 'port')
(2, 'Thirteen')
(2, 'Battle')
(2, 'their')
(2, 'base')
(2, 'national')
(2, 'hosted')
(2, 'advancements')
(2, 'ties')
(2, 'southern')
(2, 'War')
(2, "state's")
(2, 'Following')
(2, 'its')
(2, 'attacks')
(1, 'begins')
(1, 'BC,')
(1, 'when')
(1, 'Americans')
(1, '1100')
(1, 'native')
(1, 'cultu